<a href="https://colab.research.google.com/github/jeffheaton/t81_558_deep_learning/blob/master/assignments/assignment_yourname_class5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# T81-558: Applications of Deep Neural Networks
* Instructor: [Jeff Heaton](https://sites.wustl.edu/jeffheaton/), School of Engineering and Applied Science, [Washington University in St. Louis](https://engineering.wustl.edu/Programs/Pages/default.aspx)
* For more information visit the [class website](https://sites.wustl.edu/jeffheaton/t81-558/).

**Module 5 Assignment: Predicting Home Prices**

**Student Name: Hao-Tien Kuo**

# Assignment Instructions

This assignment asks you to train a neural network to predict housing prices.  I provide you with two different datasets.  You will use one data set, which includes house prices for training.  A second dataset, which does not include house prices, will be used for prediction and be submitted for evaluation.  I also give you a third dataset that contains median income for zip codes that must be joined to both the training/test datasets to provide an additional input value.  You must use the median income with your inputs for extra predictive power.

You can find all of the needed CSV files here:

* [House Prices - Training](https://data.heatonresearch.com/data/t81-558/datasets/houses_train.csv)
* [House Prices - Submit](https://data.heatonresearch.com/data/t81-558/datasets/houses_test.csv)
* [Median Income by Zipcode](https://data.heatonresearch.com/data/t81-558/datasets/zips.csv)

The median income by zipcode provides an additional feature, median income, that you should use in your predictions.  To complete this assignment perform the following steps:

* Load the housing prices training data.
* Join the median income by zipcode to the training data so that you gain the median income.
* Train a model to predict house price when given the following inputs: 'bedrooms', 'bathrooms', 'garage', 'land', 'sqft', 'median_income'.
* Load the housing prices test data.  This data does not contain the house price, you must predict this.
* Join the median income by zipcode to the test/submit data to gain the median income.
* Predict prices for the test/submit data.  
* Create a submission dataset that contains the house id (from the test/submit data) and the predicted price for that house.  Include no other fields.
* Submit this dataset and see how close you are to the actual values.

Predicting the house prices with less than +/- $10,000 is sufficient to complete the assignment.  You may also wish to see if you can get your prediction even more accurate.

A few notes/suggestions on this assignment:

* You will see really high loss rates, due to the face that the target value is the price of a house (which is a large number)
* You might want to add a RMSE metric, which is the error in the same units target value. 
* I was able to obtain an RMSE of 86; yet still had a fairly high loss of 25920990.
* To add RMSE as a metric:
```
model.compile(loss='mean_squared_error', metrics=[tf.keras.metrics.RootMeanSquaredError()], optimizer=opt)
```
# Google CoLab Instructions
If you are using Google CoLab, it will be necessary to mount your GDrive to send your notebook during the submit process. Running the following code will map your GDrive to /content/drive.

In [1]:
try:
    from google.colab import drive
    drive.mount('/content/drive', force_remount=True)
    COLAB = True
    print("Note: using Google CoLab")
    %tensorflow_version 2.x
except:
    print("Note: not using Google CoLab")
    COLAB = False

Mounted at /content/drive
Note: using Google CoLab
Colab only includes TensorFlow 2.x; %tensorflow_version has no effect.


# Assignment Submit Function

You will submit the 10 programming assignments electronically.  The following submit function can be used to do this.  My server will perform a basic check of each assignment and let you know if it sees any basic problems. 

**It is unlikely that should need to modify this function.**

In [2]:
import base64
import os
import numpy as np
import pandas as pd
import requests
import PIL
import PIL.Image
import io

# This function submits an assignment.  You can submit an assignment as much as you like, only the final
# submission counts.  The paramaters are as follows:
# data - List of pandas dataframes or images.
# key - Your student key that was emailed to you.
# no - The assignment class number, should be 1 through 1.
# source_file - The full path to your Python or IPYNB file.  This must have "_class1" as part of its name.  
# .             The number must match your assignment number.  For example "_class2" for class assignment #2.
def submit(data,key,no,source_file=None):
    if source_file is None and '__file__' not in globals(): raise Exception('Must specify a filename when a Jupyter notebook.')
    if source_file is None: source_file = __file__
    suffix = '_class{}'.format(no)
    if suffix not in source_file: raise Exception('{} must be part of the filename.'.format(suffix))
    with open(source_file, "rb") as image_file:
        encoded_python = base64.b64encode(image_file.read()).decode('ascii')
    ext = os.path.splitext(source_file)[-1].lower()
    if ext not in ['.ipynb','.py']: raise Exception("Source file is {} must be .py or .ipynb".format(ext))
    payload = []
    for item in data:
        if type(item) is PIL.Image.Image:
            buffered = BytesIO()
            item.save(buffered, format="PNG")
            payload.append({'PNG':base64.b64encode(buffered.getvalue()).decode('ascii')})
        elif type(item) is pd.core.frame.DataFrame:
            payload.append({'CSV':base64.b64encode(item.to_csv(index=False).encode('ascii')).decode("ascii")})
    r= requests.post("https://api.heatonresearch.com/assignment-submit",
        headers={'x-api-key':key}, json={ 'payload': payload,'assignment': no, 'ext':ext, 'py':encoded_python})
    if r.status_code==200:
        print("Success: {}".format(r.text))
    else: print("Failure: {}".format(r.text))

# Assignment #5 Sample Code

The following code provides a starting point for this assignment.

In [10]:
import io
import os
import requests
import numpy as np
import pandas as pd
from scipy.stats import zscore
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout
from sklearn import metrics
from sklearn.model_selection import KFold, train_test_split
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

# This is your student key that I emailed to you at the beginnning of the semester.
key = "lhkxWaPDTw1k2isuaX0Gk4X0XN0RcGFX3lEQFj0l"  # This is an example key and will not work.

# You must also identify your source file.  (modify for your local setup)
file='/content/drive/MyDrive/Colab Notebooks/Deep Neural Networks/assignment_Hao-Tien Kuo_class5.ipynb'  # Google CoLab

# Begin assignment
df_houses_train = pd.read_csv("https://data.heatonresearch.com/data/t81-558/datasets/houses_train.csv")
df_houses_submit = pd.read_csv("https://data.heatonresearch.com/data/t81-558/datasets/houses_test.csv")
df_zips = pd.read_csv("https://data.heatonresearch.com/data/t81-558/datasets/zips.csv")


#submit(source_file=file,data=[df_submit],key=key,no=5)

In [11]:
display(df_houses_train.head())
display(df_zips.head())

,id,zip,bedrooms,bathrooms,garage,land,sqft,price
0,1,60019,9,2,3,2.198,4860,1005580
1,2,60049,5,2,2,4.517,2870,620278
2,3,60011,2,1,0,4.120,1220,265711
3,4,60027,6,4,2,3.201,3810,819916
4,5,60001,9,3,2,1.347,5061,1039491


,zip,median_income
0,60000,75806
1,60001,205564
2,60002,307019
3,60003,145929
4,60004,135496


In [12]:
# Join the median income by zipcode
df_merged_train = df_houses_train.merge(df_zips, how='left', on='zip')

# Standardize ranges and covert to numpy
x_columns = df_merged_train.columns.drop('id').drop('zip').drop('price')
x = df_merged_train[x_columns].apply(zscore).values
y = df_merged_train['price'].values

# Create train/validation
x_train, x_val, y_train, y_val = train_test_split(x, y, test_size=0.2, random_state=10)

In [13]:
# Build the neural network
model = Sequential()
model.add(Dense(25, input_dim=x.shape[1], activation='relu')) # Hidden 1
model.add(Dense(10, activation='relu')) # Hidden 2
model.add(Dense(1)) # Output
model.compile(loss='mean_squared_error', metrics=[tf.keras.metrics.RootMeanSquaredError()], optimizer='adam')
monitor = EarlyStopping(monitor='val_loss', min_delta=1e-3, 
                        patience=5, verbose=1, mode='auto', 
                        restore_best_weights=True)
model.fit(x_train,y_train,validation_data=(x_val,y_val),
          callbacks=[monitor],verbose=2,epochs=1000)

Epoch 1/1000
250/250 - 1s - loss: 490027122688.0000 - root_mean_squared_error: 700019.3750 - val_loss: 484876517376.0000 - val_root_mean_squared_error: 696330.7500 - 1s/epoch - 5ms/step
Epoch 2/1000
250/250 - 1s - loss: 489919545344.0000 - root_mean_squared_error: 699942.5000 - val_loss: 484664049664.0000 - val_root_mean_squared_error: 696178.1875 - 571ms/epoch - 2ms/step
Epoch 3/1000
250/250 - 1s - loss: 489545924608.0000 - root_mean_squared_error: 699675.5625 - val_loss: 484118822912.0000 - val_root_mean_squared_error: 695786.5000 - 528ms/epoch - 2ms/step
Epoch 4/1000
250/250 - 1s - loss: 488776269824.0000 - root_mean_squared_error: 699125.3750 - val_loss: 483134734336.0000 - val_root_mean_squared_error: 695078.9375 - 531ms/epoch - 2ms/step
Epoch 5/1000
250/250 - 1s - loss: 487511523328.0000 - root_mean_squared_error: 698220.2500 - val_loss: 481614069760.0000 - val_root_mean_squared_error: 693984.1875 - 553ms/epoch - 2ms/step
Epoch 6/1000
250/250 - 1s - loss: 485671796736.0000 - root

In [15]:
# Predict
pred = model.predict(x_val)

# Measure MSE error
score = metrics.mean_squared_error(pred,y_val)
print("Final score (MSE): {}".format(score))

# Measure RMSE error.  RMSE is common for regression.
score = np.sqrt(metrics.mean_squared_error(pred,y_val))
print("Final score (RMSE): {}".format(score))

Final score (MSE): 542.5206775817871
Final score (RMSE): 23.292073277872607


In [18]:
# Join the median income by zipcode
df_merged_submit = df_houses_submit.merge(df_zips, how='left', on='zip')

# Standardize ranges and covert to numpy
x_submit = df_merged_submit[x_columns].apply(zscore).values

In [27]:
# Predict
pred_submit = pd.DataFrame(model.predict(x_submit), columns=['price'])
df_submit = pd.concat([df_houses_submit['id'], pred_submit], axis=1)

In [28]:
submit(source_file=file,data=[df_submit],key=key,no=5)

Success: Submitted Assignment 5 for h.kuo:
You have submitted this assignment 2 times. (this is fine)
Note: The mean difference 4817.925047499943 for column 'price' is acceptable and is less than the maximum allowed value of '10000' for this assignment.
No warnings or errors (only notes), you will probably do well, but no guarantee. :-)
